In [1]:
%matplotlib inline

In [2]:
import numpy as np

class MF():

    def __init__(self, R, K, alpha, beta, iterations, test_samples, X=None, Y=None, Z=None, src_si_len=0, \
                 tgt_si_len=0, lang_pair_si_len=0, src_index=None, tgt_index=None, model=None, num_running=0):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)                  : src-tgt language rating matrix
        - K (int)                      : number of latent dimensions
        - alpha (float)                : learning rate
        - beta (float)                 : regularization parameter
        - X (dict)                     : source language side information
        - Y (dict)                     : target language side information
        - Z (dict)                     : language pair side information
        - src_si_len(int)              : source language side information length
        - tgt_si_len(int)              : target language side information length
        - lang_pair_si_len(int)        : language pair side information length
        """

        self.R = np.array(R)
        self.Prediction = deepcopy(self.R)
        self.src_langs = R.index.tolist()
        self.tgt_langs = R.columns.tolist()
        self.num_src, self.num_tgt = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.X = X
        self.Y = Y
        self.Z = Z
        self.src_si_len = src_si_len
        self.tgt_si_len = tgt_si_len
        self.lang_pair_si_len = lang_pair_si_len
        self.test_samples = test_samples
        self.src_index=src_index 
        self.tgt_index=tgt_index
        self.model=model
        self.num_running = num_running
        self.score_dict = {'BLEU': "WIKI-MT", "Muse": "BLI-Muse", "Vecmap": "BLI-Vecmap"}
        self.traing_error_log=[]
        self.test_error_log=[]
        

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_src, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_tgt, self.K))

        # Initialize side information's parameter if necesary
        if self.X and self.src_si_len:
            self.A = np.random.normal(scale = 1./self.src_si_len, size=self.src_si_len)
        if self.Y and self.tgt_si_len:
            self.B = np.random.normal(scale = 1./self.tgt_si_len, size=self.tgt_si_len)
        if self.Z and self.lang_pair_si_len:
            self.C = np.random.normal(scale = 1./self.lang_pair_si_len, size=self.lang_pair_si_len)
        
        # Initialize the biases
        # the biases of users and items are initilized as 0
        # the bias of rating is initilized as mean value
        self.b_u = np.zeros(self.num_src)
        self.b_i = np.zeros(self.num_tgt)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples (where rating > 0)
        self.samples = []
        for i in range(self.num_src):
            for j in range(self.num_tgt):
                if self.R[i,j] > 0:
                    cur_tuple = [i, j, self.R[i, j]]
                    src_lang = self.src_langs[i]
                    tgt_lang = self.tgt_langs[j]
                    if self.X:
                        if src_lang in self.X.keys():
                            cur_tuple.append(self.X[src_lang])
                        else:
                            raise KeyError
                    if self.Y:
                        if tgt_lang in self.Y.keys():
                            cur_tuple.append(self.Y[src_lang])
                        else:
                            raise KeyError
                    if self.Z:
                        if src_lang + "_" + tgt_lang in self.Z.keys():
                            cur_tuple.append(self.Z[src_lang + "_" + tgt_lang])
                        else:
                            raise KeyError
                    self.samples.append(tuple(cur_tuple))

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            # shuffle training samples
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 100 == 0:
                self.traing_error_log.append((i, mse))
                test_mse = self.evaluate_testing(self.test_samples, self.src_index, self.tgt_index, self.model)
                self.test_error_log.append((i, test_mse))
#                 print("\t\tIteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
#         predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x][y] - self.Prediction[x][y], 2)
        return np.sqrt(error / len(xs))

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for sample in self.samples:
            i, j, r = sample[0], sample[1], sample[2]
            # Computer prediction and error
            prediction = self.get_rating(sample)
            self.Prediction[i][j] = prediction
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
            
            # Update side information parameter if necessary
            cur_index = 3
            if self.X:
                x = np.array(sample[cur_index], dtype=np.float64)
                cur_index += 1
                self.A += self.alpha * (e * x - self.beta * self.A)
            if self.Y:
                y = np.array(sample[cur_index], dtype=np.float64)
                cur_index += 1
                self.B += self.alpha * (e * y - self.beta * self.B)
            if self.Z:
                z = np.array(sample[cur_index], dtype=np.float64)
                self.C += self.alpha * (e * z - self.beta * self.C)
                

    def get_rating(self, sample):
        """
        Get the predicted rating of sample
        """
        i, j, r = sample[0], sample[1], sample[2]
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        cur_index = 3
        if self.X:
            x = sample[cur_index]
            cur_index += 1
            prediction += self.A.dot(x.T)
        if self.Y:
            y = sample[cur_index]
            cur_index += 1
            prediction += self.B.dot(y.T)
        if self.Z:
            z = sample[cur_index]
            prediction += self.C.dot(z.T)
        return prediction
    
    def evaluate_testing(self, test_data, src_index_name, tgt_index_name, score_index_name):
        """
        Predict the score for testing data
        """ 
        rmse = 0.0
        for record in test_data.iterrows():
            record = record[1]
            src_lang = record[src_index_name]
            tgt_lang = record[tgt_index_name]
            src_lang_index = self.src_langs.index(src_lang)
            tgt_lang_index = self.tgt_langs.index(tgt_lang)
            score = record[score_index_name]
            cur_tuple = [src_lang_index, tgt_lang_index, score]
            if self.X:
                if src_lang in self.X.keys():
                    cur_tuple.append(self.X[src_lang])
                else:
                    raise KeyError
            if self.Y:
                if tgt_lang in self.Y.keys():
                    cur_tuple.append(self.Y[src_lang])
                else:
                    raise KeyError
            if self.Z:
                if src_lang + "_" + tgt_lang in self.Z.keys():
                    cur_tuple.append(self.Z[src_lang + "_" + tgt_lang])
                else:
                    raise KeyError
            prediction = self.get_rating(tuple(cur_tuple))
            rmse += (prediction - score) * (prediction - score)
        return np.sqrt(rmse / len(test_data))
        

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P, Q, A, B, and C
        """
        res = deepcopy(self.R)
        for i in range(self.num_src):
            for j in range(self.num_tgt):
                src_lang = self.src_langs[i]
                tgt_lang = self.tgt_langs[j]
                res[i][j] = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
                if self.X and src_lang in self.X.keys():
                    x = self.X[src_lang]
                    res[i][j] += self.A.dot(x.T)
                if self.Y and tgt_lang in self.Y.keys():
                    y = self.X[tgt_lang]
                    res[i][j] += self.A.dot(x.T)
                if self.Z and src_lang + "_" + tgt_lang in self.Z.keys():
                    z = self.X[src_lang + "_" + tgt_lang]
                    res[i][j] += self.A.dot(x.T)
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)
    
    def draw_error_curve(self, i):
        import matplotlib.pyplot as plt
        import numpy as np
        iters = []
        train_loss = []
        test_loss = []
        for item in self.traing_error_log:
            iters.append(item[0])
            train_loss.append(item[1])
        for item in self.test_error_log:
            test_loss.append(item[1])

        plt.plot(iters, train_loss, 'b', label='train loss')#'b'指：color='blue'
        plt.plot(iters, test_loss, 'r', label='test loss')#'r'指：color='red'

        plt.legend()  #显示上面的label
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        title = self.score_dict[self.model] + "_" + str(self.num_running+1) + "_Fold" + str(i+1)
        plt.title(title) 
        plt.savefig("../result/within_pair_si/" + self.score_dict[self.model] + "/fold" + str(i+1) + "/" + title + ".png")

        #plt.ylim(-1,1)#仅设置y轴坐标范围
        plt.show()

        

Represent the two tasks above as matrices (BLEU scores for Wiki-MT and Accuracy for BLI) 

In [3]:
import pandas as pd
import numpy as np
np.random.seed(2021)

In [4]:
from copy import deepcopy
import pandas as pd
import random as rd

def split_k_fold_data(file_dir, score_index_name, src_index_name, tgt_index_name, k=5):
    data = pd.read_csv(file_dir)
    
    # shuffle
    data = data.sample(frac=1)
    
    # generate score matrix
    src_langs = data[src_index_name].unique()
    tgt_langs = data[tgt_index_name].unique()
    score_matrix = pd.DataFrame(index = src_langs, columns = tgt_langs)
    
    # eliminate empty rows and columns
    data = data.dropna(axis=1, how="all")
    data = data.dropna(axis=0, how="all")
    
    # K fold split
    k_fold_data = {}
    models = list(score_index_name)
    lens= len(data)
    
    for i, model in enumerate(models):
        ex_per_fold = int(np.ceil(lens / k))
        for j in range(k):
            start = ex_per_fold * j
            end = ex_per_fold * (j + 1)
            if j == 0:
                k_fold_data[model] = {"train": [], "test": []}
            k_fold_data[model]["train"].append(pd.concat([data.iloc[:start, :], data.iloc[end:, :]], axis=0))
            k_fold_data[model]["test"].append(data.iloc[start:end, :])
    return k_fold_data, score_matrix

def generate_score_matrix(train_data, src_index_name, tgt_index_name, score_index_name, origin_score_matrix):    
    score_matrix = deepcopy(origin_score_matrix)
    
    for record in train_data.iterrows():
        record = record[1]
        src_lang = record[src_index_name]
        tgt_lang = record[tgt_index_name]
        score = record[score_index_name]
        score_matrix.loc[src_lang, tgt_lang] = score
#         score_matrix[src_lang][tgt_lang] = score
    score_matrix.fillna(0, inplace=True)
    return score_matrix

def get_rmse(valid_data, model, src_index_name, tgt_index_name, score_matrix, train_matrix):
    rmse = 0.0
    src_langs = train_matrix.index.tolist()
    tgt_langs = train_matrix.columns.tolist()
    for cur_valid_data in valid_data.iterrows():
        cur_valid_data = cur_valid_data[1]
        src_lang, tgt_lang, score = cur_valid_data[src_index_name], cur_valid_data[tgt_index_name], cur_valid_data[model]
        src_idx = src_langs.index(src_lang)
        tgt_idx = tgt_langs.index(tgt_lang)
        prediction = score_matrix[src_idx][tgt_idx]
        rmse += (prediction - score) * (prediction - score)
    return np.sqrt(rmse / len(valid_data))

def get_result(alpha, beta, data_dir, scores, src_index, tgt_index, k, num_running, src_lang_side_info=None, \
               tgt_lang_side_info=None, lang_pair_side_info=None, src_si_len=0, tgt_si_len=0, \
               lang_pair_si_len=0):
    all_running_rmse = 0.0
    for nr in range(num_running):
        print("num running: " + str(nr+1))
        data, langs_matrix = split_k_fold_data(data_dir, scores, src_index, tgt_index, k)
        res = {}
        for model in scores:
            print("-"*40)
            print(model)
            total_rmse = 0.0
            for i in range(k):
                print("\tFold {}: ".format(i+1))
                train_data, test_data = data[model]["train"][i], data[model]["test"][i]
                train_matrix = generate_score_matrix(train_data, src_index, tgt_index, model, langs_matrix)
                mf = MF(train_matrix, K=2, alpha=alpha, beta=beta, iterations=2000, test_samples=test_data, X=src_lang_side_info, \
                        Y=tgt_lang_side_info, Z=lang_pair_side_info, src_si_len=src_si_len, tgt_si_len=tgt_si_len, \
                        lang_pair_si_len=lang_pair_si_len, src_index=src_index, tgt_index=tgt_index, model=model, \
                       num_running=nr)
                trainging_log = mf.train()
#                 predictions = mf.full_matrix()
#                 cur_rmse = get_rmse(test_data, model, src_index, tgt_index, predictions, train_matrix)
                cur_rmse = mf.evaluate_testing(test_data, src_index, tgt_index, model)
#                 mf.draw_error_curve(i)
                total_rmse += cur_rmse
                print("\t\trmse is {}.".format(cur_rmse))
                print("*" * 20)
                
            average_rmse = total_rmse / k
            print("average rmse: " + str(average_rmse))
            res[model] = average_rmse
        res_rmse = 0.0
        for key, value in res.items():
            res_rmse += value
        all_running_rmse += res_rmse / len(scores)
    return all_running_rmse / num_running

def get_language_pair_side_info(data_dir, side_info_features, src_lang_name, tgt_lang_name):
    data = pd.read_csv(data_dir)
    side_dict = {}
    for record in data.iterrows():
        record = record[1]
        src_lang = record[src_lang_name]
        tgt_lang = record[tgt_lang_name]
        side_dict[src_lang + "_" + tgt_lang] = record[side_info_features].values
    return side_dict

In [5]:
WIKI_MT_DIR = "../data/data_wiki.csv"
WIKI_SRC = "Source"
WIKI_TGT = "Target"
WIKI_SCORE = ["BLEU"]
WIKI_SIDE_FEATURES = ['geographic', 'genetic', 'inventory', 'syntactic', 'phonological', 'featural']

BLI_DIR = "../data/data_bli2.csv"
BLI_SRC = "Source Language Code"
BLI_TGT = "Target Language Code"
BLI_SCORE = ["Muse", "Vecmap"]
BLI_SIDE_FEATURES = ['genetic', 'syntactic', 'featural', 'phonological', 'inventory', 'geographic']
# The score metric is from "NLPerf/src/task_feats.py"

k = 5
num_running = 10

WIKI_SIDE_INFO_DICT = get_language_pair_side_info(WIKI_MT_DIR, WIKI_SIDE_FEATURES, WIKI_SRC, WIKI_TGT)
BLI_SIDE_INFO_DICT = get_language_pair_side_info(BLI_DIR, BLI_SIDE_FEATURES, BLI_SRC, BLI_TGT)


candidate_alpha = [0.001, 0.0005, 0.0001]
candidate_beta = [0.1, 0.01, 0.001, 0.0005]

gs_result = {}

for alpha in candidate_alpha:
    for beta in candidate_beta:

        wiki_result = get_result(alpha, beta, WIKI_MT_DIR, WIKI_SCORE, WIKI_SRC, WIKI_TGT, k, num_running, \
                                 lang_pair_side_info = WIKI_SIDE_INFO_DICT, lang_pair_si_len = 6)
        bli_result = get_result(alpha, beta, BLI_DIR, BLI_SCORE, BLI_SRC, BLI_TGT, k, num_running, \
                         lang_pair_side_info = BLI_SIDE_INFO_DICT, lang_pair_si_len = 6)
        gs_result[str(alpha) + "_" + str(beta)] = [wiki_result, bli_result]

# wiki_mt_score_matrix = generate_score_matrix(WIKI_MT_DIR, "Source", "Target", "BLEU")
# wiki_mt_score_matrix, wiki_mt_valid = generate_score_matrix(WIKI_MT_DIR, "Source", "Target", "BLEU")xdz
# bli_score_matrix, bli_valid = generate_score_matrix(BLI_DIR, "Source Language Code", "Target Language Code", ["Muse", "Vecmap"])

num running: 1
----------------------------------------
BLEU
	Fold 1: 
		rmse is 4.485801698741001.
********************
	Fold 2: 
		rmse is 3.296638539387891.
********************
	Fold 3: 
		rmse is 2.6550927905205612.
********************
	Fold 4: 
		rmse is 3.5645542624187443.
********************
	Fold 5: 
		rmse is 2.247707898237903.
********************
average rmse: 3.2499590378612195
num running: 2
----------------------------------------
BLEU
	Fold 1: 
		rmse is 3.067354842211844.
********************
	Fold 2: 
		rmse is 2.191151481697373.
********************
	Fold 3: 
		rmse is 2.9704444499878604.
********************
	Fold 4: 
		rmse is 2.3912987467163123.
********************
	Fold 5: 
		rmse is 3.420010257141738.
********************
average rmse: 2.8080519555510257
num running: 3
----------------------------------------
BLEU
	Fold 1: 
		rmse is 3.5815332159707776.
********************
	Fold 2: 
		rmse is 3.6748320649341806.
********************
	Fold 3: 
		rmse is 3.177

		rmse is 7.595917613834901.
********************
	Fold 2: 
		rmse is 8.536667151558484.
********************
	Fold 3: 
		rmse is 8.300609934495325.
********************
	Fold 4: 
		rmse is 12.145430156352381.
********************
	Fold 5: 
		rmse is 27.566895770838485.
********************
average rmse: 12.829104125415915
num running: 7
----------------------------------------
Muse
	Fold 1: 
		rmse is 14.081684606811828.
********************
	Fold 2: 
		rmse is 7.616016348725325.
********************
	Fold 3: 
		rmse is 14.169626658138135.
********************
	Fold 4: 
		rmse is 22.731589581917607.
********************
	Fold 5: 
		rmse is 11.608196508669328.
********************
average rmse: 14.041422740852443
----------------------------------------
Vecmap
	Fold 1: 
		rmse is 13.18781495806207.
********************
	Fold 2: 
		rmse is 6.077766903618174.
********************
	Fold 3: 
		rmse is 9.09315720970451.
********************
	Fold 4: 
		rmse is 8.525803078387538.
***********

		rmse is 16.173441984499647.
********************
	Fold 2: 
		rmse is 7.716024972675601.
********************
	Fold 3: 
		rmse is 8.17579380939822.
********************
	Fold 4: 
		rmse is 10.96115804337805.
********************
	Fold 5: 
		rmse is 9.379726086737708.
********************
average rmse: 10.481228979337846
----------------------------------------
Vecmap
	Fold 1: 
		rmse is 8.865795647321061.
********************
	Fold 2: 
		rmse is 8.472513827775717.
********************
	Fold 3: 
		rmse is 6.1493428230383245.
********************
	Fold 4: 
		rmse is 8.41813183835844.
********************
	Fold 5: 
		rmse is 15.423737945184486.
********************
average rmse: 9.465904416335606
num running: 3
----------------------------------------
Muse
	Fold 1: 
		rmse is 45.62155789401902.
********************
	Fold 2: 
		rmse is 9.256362843691328.
********************
	Fold 3: 
		rmse is 22.444409712152066.
********************
	Fold 4: 
		rmse is 47.113458638856834.
**************

		rmse is 6.588320416439075.
********************
	Fold 2: 
		rmse is 3.5936692794708858.
********************
	Fold 3: 
		rmse is 3.001481457733529.
********************
	Fold 4: 
		rmse is 2.5073101561981836.
********************
	Fold 5: 
		rmse is 2.3893358728054026.
********************
average rmse: 3.616023436529415
num running: 5
----------------------------------------
BLEU
	Fold 1: 
		rmse is 2.7983387712788748.
********************
	Fold 2: 
		rmse is 2.995907865241163.
********************
	Fold 3: 
		rmse is 2.6663722571684882.
********************
	Fold 4: 
		rmse is 12.123960093414672.
********************
	Fold 5: 
		rmse is 5.939784148457702.
********************
average rmse: 5.30487262711218
num running: 6
----------------------------------------
BLEU
	Fold 1: 
		rmse is 5.519868154541379.
********************
	Fold 2: 
		rmse is 2.570964032865074.
********************
	Fold 3: 
		rmse is 2.8693083069663805.
********************
	Fold 4: 
		rmse is 2.4177511006869836

		rmse is 14.5785552468208.
********************
	Fold 2: 
		rmse is 13.660178082291011.
********************
	Fold 3: 
		rmse is 10.109015678021567.
********************
	Fold 4: 
		rmse is 7.059212942515647.
********************
	Fold 5: 
		rmse is 11.793030587373389.
********************
average rmse: 11.439998507404482
----------------------------------------
Vecmap
	Fold 1: 
		rmse is 14.269931624916111.
********************
	Fold 2: 
		rmse is 7.129968220654123.
********************
	Fold 3: 
		rmse is 7.657129398221493.
********************
	Fold 4: 
		rmse is 8.040572260144566.
********************
	Fold 5: 
		rmse is 19.755942415278923.
********************
average rmse: 11.370708783843044
num running: 9
----------------------------------------
Muse
	Fold 1: 
		rmse is 10.148299635487342.
********************
	Fold 2: 
		rmse is 9.022202775633774.
********************
	Fold 3: 
		rmse is 10.128199089507195.
********************
	Fold 4: 
		rmse is 8.72077530916674.
***********

...

		rmse is 3.657850939578427.
********************
	Fold 3: 
		rmse is 3.107945359892544.
********************
	Fold 4: 
		rmse is 2.581215711869562.
********************
	Fold 5: 
		rmse is 2.819760613983342.
********************
average rmse: 2.9883561856856256
num running: 5
----------------------------------------
BLEU
	Fold 1: 
		rmse is 2.5962015344997544.
********************
	Fold 2: 
		rmse is 3.2990738778580684.
********************
	Fold 3: 
		rmse is 2.5160587581050597.
********************
	Fold 4: 
		rmse is 2.6893508827199364.
********************
	Fold 5: 
		rmse is 2.9800008415673096.
********************
average rmse: 2.816137178950026
num running: 6
----------------------------------------
BLEU
	Fold 1: 
		rmse is 2.7331475247632375.
********************
	Fold 2: 
		rmse is 3.458244052647046.
********************
	Fold 3: 
		rmse is 3.027787409428614.
********************
	Fold 4: 
		rmse is 3.0233075253067616.
********************
	Fold 5: 
		rmse is 3.19604401663640

		rmse is 8.144662048395478.
********************
	Fold 3: 
		rmse is 8.42841977437223.
********************
	Fold 4: 
		rmse is 17.89231377683993.
********************
	Fold 5: 
		rmse is 8.410400907384169.
********************
average rmse: 11.11460733795862
----------------------------------------
Vecmap
	Fold 1: 
		rmse is 12.597680792379723.
********************
	Fold 2: 
		rmse is 10.611545742104376.
********************
	Fold 3: 
		rmse is 9.803605070949548.
********************
	Fold 4: 
		rmse is 12.954160809744163.
********************
	Fold 5: 
		rmse is 6.59856199082573.
********************
average rmse: 10.513110881200708
num running: 9
----------------------------------------
Muse
	Fold 1: 
		rmse is 8.79214161806966.
********************
	Fold 2: 
		rmse is 15.865485292417446.
********************
	Fold 3: 
		rmse is 8.662352694351963.
********************
	Fold 4: 
		rmse is 20.17697063862274.
********************
	Fold 5: 
		rmse is 12.847303952718658.
***************

		rmse is 15.246942114366732.
********************
	Fold 3: 
		rmse is 10.177203387791353.
********************
	Fold 4: 
		rmse is 11.20629077313143.
********************
	Fold 5: 
		rmse is 5.947430680853158.
********************
average rmse: 11.22798973606391
num running: 4
----------------------------------------
Muse
	Fold 1: 
		rmse is 16.11933152634564.
********************
	Fold 2: 
		rmse is 11.25034617576219.
********************
	Fold 3: 
		rmse is 10.438676639913654.
********************
	Fold 4: 
		rmse is 12.93207791503269.
********************
	Fold 5: 
		rmse is 8.35364686392584.
********************
average rmse: 11.818815824196001
----------------------------------------
Vecmap
	Fold 1: 
		rmse is 10.296726594438141.
********************
	Fold 2: 
		rmse is 11.76309394925309.
********************
	Fold 3: 
		rmse is 8.528997625942814.
********************
	Fold 4: 
		rmse is 10.15763366618511.
********************
	Fold 5: 
		rmse is 10.438320753653436.
*************

		rmse is 3.2166480437974005.
********************
	Fold 3: 
		rmse is 2.946725950320826.
********************
	Fold 4: 
		rmse is 3.955884349766468.
********************
	Fold 5: 
		rmse is 2.2951982825101447.
********************
average rmse: 3.1191954938702904
num running: 8
----------------------------------------
BLEU
	Fold 1: 
		rmse is 3.0350437804015735.
********************
	Fold 2: 
		rmse is 3.2626370560559557.
********************
	Fold 3: 
		rmse is 2.795471955272072.
********************
	Fold 4: 
		rmse is 2.5411074807764265.
********************
	Fold 5: 
		rmse is 3.259221918222137.
********************
average rmse: 2.978696438145633
num running: 9
----------------------------------------
BLEU
	Fold 1: 
		rmse is 2.2439208251802505.
********************
	Fold 2: 
		rmse is 3.375444339424622.
********************
	Fold 3: 
		rmse is 3.9239651910572184.
********************
	Fold 4: 
		rmse is 2.974524179680113.
********************
	Fold 5: 
		rmse is 2.46629131861392

		rmse is 11.699084656674158.
********************
	Fold 3: 
		rmse is 10.430318113882832.
********************
	Fold 4: 
		rmse is 9.746210905138613.
********************
	Fold 5: 
		rmse is 8.974600745789335.
********************
average rmse: 10.172775014731835
num running: 10
----------------------------------------
Muse
	Fold 1: 
		rmse is 14.176950522185065.
********************
	Fold 2: 
		rmse is 15.656800643126278.
********************
	Fold 3: 
		rmse is 10.068462286780699.
********************
	Fold 4: 
		rmse is 9.945590787880096.
********************
	Fold 5: 
		rmse is 10.948997284801003.
********************
average rmse: 12.159360304954628
----------------------------------------
Vecmap
	Fold 1: 
		rmse is 13.374925640362164.
********************
	Fold 2: 
		rmse is 13.1017022822258.
********************
	Fold 3: 
		rmse is 10.334282413499102.
********************
	Fold 4: 
		rmse is 9.750245627652044.
********************
	Fold 5: 
		rmse is 8.070935741884396.
********

In [6]:
print(wiki_result)
print(bli_result)

3.002674345179626
11.033629872939725


In [7]:
gs_result

{'0.001_0.1': [3.1146016219612287, 13.4443412275802],
 '0.001_0.01': [3.7083219809630954, 14.581027128180915],
 '0.001_0.001': [3.969528042697438, 12.614980050620767],
 '0.001_0.0005': [3.7310301838872824, 15.07697849134693],
 '0.0005_0.1': [3.147650515925009, 13.845901548087587],
 '0.0005_0.01': [3.582871934660725, 13.437873844167985],
 '0.0005_0.001': [3.3108654900017633, 12.097018537894943],
 '0.0005_0.0005': [3.2205053694713066, 12.715056466188608],
 '0.0001_0.1': [3.1321937394474704, 12.122515707878977],
 '0.0001_0.01': [3.045973143783196, 11.376328397205308],
 '0.0001_0.001': [3.0096930515281386, 11.598180506478164],
 '0.0001_0.0005': [3.002674345179626, 11.033629872939725]}